In [4]:
import numpy as np
import pandas as pd
import json
import requests
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.image import ContainerImage
from azureml.core.experiment import Experiment
from azureml.core import Workspace, Dataset


In [3]:
ws = Workspace.from_config()

In [ ]:
webservice_name = 'turbofan-rul-service'

In [ ]:
dataset = Dataset.get_by_name(ws, name='NASA TurboFan Raw')
df_pd = dataset.to_pandas_dataframe()

In [ ]:
service = Webservice(ws, webservice_name) 

uri = service.scoring_uri
key =service.get_keys()[0]

print(uri)

sampleSize = round(2/len(train),4)
testdf = df_pd.sample(frac=sampleSize)

X = testdf.drop('rul',axis=1)
y = testdf.rul.tolist()


input_data = json.dumps(X.to_json( orient='records')) # Create JSON of sample records    

headers = {'Content-Type':'application/json'}

#for AKS deployment you'd need to the service key in the header as well    
headers = {'Content-Type':'application/json',  'Authorization':('Bearer '+ key)} 


resp = requests.post(uri, input_data, headers=headers)
try:
    print("POST to url", uri)
    print("input data:", input_data)
    print("label:", y)
    print("prediction:", resp.text)
except Exception:
    print("Web Service Not Found")